In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :4]

        #past_candles = []
        #for z in range(1, how_many_past_candles+1):
        #  past_candles.append(data[i-z][3])
        #past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append(data[i+z][3]-data[i+z][0])
        next_candles = sum(next_candles)
        if next_candles > 0:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close"])
        del rows
        del next_candles
        dfs.append(x)
        del x
    del data

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                           scale_padding=0.2, returnfig=True, savefig=address, tight_layout=True,figsize =(3,2),)
        
        plt.close("all")
        del df
        del date_list
        del rrng
        del i
        del right_now
        gc.collect()



        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 160 == 0:
          print(" ")

    print("")
    del dfs
    del suggs
    del index
    del all_n
    gc.collect()



def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["sony","now","adyey","shop","msi","dell","zen"]
download_data(symbols,"300d","1d")

Symbols : 1500
Data Folder Removed
 


In [4]:
how_many_future_candles = 7
how_many_past_candles = 7
each_row_past = 120

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 7
Processing File:
File Number 1: 10/173 20/173 30/173 40/173 50/173 60/173 70/173 80/173 90/173 100/173 110/173 120/173 130/173 140/173 150/173 160/173  
170/173 
File Number 2: 10/173 20/173 30/173 40/173 50/173 60/173 70/173 80/173 90/173 100/173 110/173 120/173 130/173 140/173 150/173 160/173  
170/173 
File Number 3: 10/173 20/173 30/173 40/173 50/173 60/173 70/173 80/173 90/173 100/173 110/173 120/173 130/173 140/173 150/173 160/173  
170/173 
File Number 4: 10/173 20/173 30/173 40/173 50/173 60/173 70/173 80/173 90/173 100/173 110/173 120/173 130/173 140/173 150/173 160/173  
170/173 
File Number 5: 10/173 20/173 30/173 40/173 50/173 60/173 70/173 80/173 90/173 100/173 110/173 120/173 130/173 140/173 150/173 160/173  
170/173 
File Number 6: 10/173 20/173 30/173 40/173 50/173 60/173 70/173 80/173 90/173 100/173 110/173 120/173 130/173 140/173 150/173 160/173  
170/173 
File Number 7: 10/173 20/173 30/173 40/173 50/173 60/173 70/173 80/173 90/173 100/173 110/173 1

1211

In [5]:
#folder_name = '122227'



label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name)
    dim = (300, 200)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255
print(data.shape)
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.2,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

(1211, 200, 300, 3)
xTrain : 968 \ xTest : 243
yn: 527 nn: 684


In [8]:
model = Sequential()

model.add(Conv2D(128,   (3, 3),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],3)))
model.add(MaxPooling2D(pool_size=(3, 2)))
model.add(Conv2D(128,  (3, 3),activation="relu",))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64,  (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(900,activation="relu"))
model.add(Dense(900,activation="relu"))
model.add(Dense(900,activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 198, 298, 128)     3584      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 66, 149, 128)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 147, 128)      147584    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 21, 49, 128)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 20, 48, 64)        32832     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 16, 64)       

In [10]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(xTrain,yTrain,batch_size=64,epochs=30,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)


Epoch 1/30
16/16 [==============================] - 8s 366ms/step - loss: 0.0038 - accuracy: 0.9990 - val_loss: 0.6798 - val_accuracy: 0.8519
Epoch 2/30
16/16 [==============================] - 4s 274ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.7877 - val_accuracy: 0.8436
Epoch 3/30
16/16 [==============================] - 4s 276ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.8534 - val_accuracy: 0.8395
Epoch 4/30
16/16 [==============================] - 4s 277ms/step - loss: 5.8412e-04 - accuracy: 1.0000 - val_loss: 0.8925 - val_accuracy: 0.8436
Epoch 5/30
16/16 [==============================] - 4s 277ms/step - loss: 4.2477e-04 - accuracy: 1.0000 - val_loss: 0.9188 - val_accuracy: 0.8477
Epoch 6/30
16/16 [==============================] - 4s 277ms/step - loss: 3.4788e-04 - accuracy: 1.0000 - val_loss: 0.9413 - val_accuracy: 0.8477
Epoch 7/30
16/16 [==============================] - 4s 277ms/step - loss: 2.9364e-04 - accuracy: 1.0000 - val_loss: 0.9608 - val_accurac

In [90]:
shutil.rmtree("/content/checkpoints/")